In [548]:
pd.options.display.max_rows    = 999
pd.options.display.max_columns = 999

In [549]:
from sklearn.preprocessing import LabelEncoder
def label_encoding(df_train, df_valid, df_test):
    
    factors = df_train.select_dtypes('object').columns
    
    lbl = LabelEncoder()

    for f in factors:        
        lbl.fit(list(df_train[f].values) + list(df_valid[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_valid[f] = lbl.transform(list(df_valid[f].values))
        df_test[f]  = lbl.transform(list(df_test[f].values))

    return df_train, df_valid, df_test

In [550]:
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.linear_model import LogisticRegression
import numpy as np
import time
from sklearn.metrics import log_loss, roc_auc_score, confusion_matrix

In [551]:
files = sorted(os.listdir("../oof_preds"))
files.remove('.ipynb_checkpoints')

In [552]:
files

['09072.csv',
 '4_times17_53_48_44_43_9074.csv',
 '4_times17_53_48_9068.csv',
 'lgb_v11_90707.csv',
 'lgb_v12_90622.csv',
 'lgb_v14_90605.csv',
 'lgb_v15_90624.csv',
 'lgb_v17_90519.csv',
 'lgb_v18_90494.csv',
 'lgb_v1_90545.csv',
 'lgb_v20_90522.csv',
 'lgb_v21_90535.csv',
 'lgb_v22_90501.csv',
 'lgb_v23_90548.csv',
 'lgb_v24_90467.csv',
 'lgb_v26_90633.csv',
 'lgb_v28_90684.csv',
 'lgb_v29_90658.csv',
 'lgb_v30_90492.csv',
 'lgb_v32_90494.csv',
 'lgb_v33_90234.csv',
 'lgb_v34_90561.csv',
 'lgb_v35_90553.csv',
 'lgb_v36_90560.csv',
 'lgb_v37_90526.csv',
 'lgb_v38_90545.csv',
 'lgb_v3_90547.csv',
 'lgb_v40_90501.csv',
 'lgb_v41_90534.csv',
 'lgb_v42_90540.csv',
 'lgb_v43_90603.csv',
 'lgb_v44_90592.csv',
 'lgb_v45_90523.csv',
 'lgb_v46_90349.csv',
 'lgb_v47_90384.csv',
 'lgb_v48_seed111_90581.csv',
 'lgb_v49_seed111_90541.csv',
 'lgb_v49_seed111_90570.csv',
 'lgb_v4_90630.csv',
 'lgb_v50_seed111_90566.csv',
 'lgb_v51_seed111_90558.csv',
 'lgb_v52_seed111_90564.csv',
 'lgb_v53_seed111_9

In [553]:
files = ['09072.csv','lgb_v17_90519.csv','lgb_v73_seed111_90648.csv',
'lgb_v80_seed111_90642.csv','lgb_v84_seed111_90668.csv','lgb_v92_seed111_90701.csv',]

In [554]:
train_ = pd.read_csv('../raw/training.csv')
train_ = train_[-train_['hospital_death'].isnull()]
train_.shape

/home/RDC/zinovyee.hub/.conda/envs/wids/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (7,8,10,11,13,14,184,185) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(91713, 186)

In [555]:
train = pd.DataFrame( columns=files)
test  = pd.DataFrame( columns=files)

In [556]:
y = train_['hospital_death']

In [557]:
for i, file in enumerate(files):
    predictions = pd.read_csv(f'../oof_preds/{file}')
    if file == files[0]:
        train['encounter_id'] = predictions['encounter_id']
    train[file] = predictions['hospital_death']

In [558]:
for i, file in enumerate(files):
    predictions = pd.read_csv(f'../submissions/{file}')
    if file == files[0]:
        test['encounter_id'] = predictions['encounter_id']
    test[file] = predictions['hospital_death']

In [559]:
excluded_feats = ['encounter_id']
features = [f for f in train.columns if f not in excluded_feats]

### STACKING

In [560]:
oof_preds = train[features].mean(axis=1)
preds_test = test[features].mean(axis=1).values

In [561]:
from scipy.stats import pearsonr

In [562]:
pearsonr(preds_test, test['lgb_v17_90519.csv'])

(0.9927119709406216, 0.0)

In [563]:
pearsonr(oof_preds, train['lgb_v17_90519.csv'])

(0.9881105659714902, 0.0)

In [564]:
oof = pd.DataFrame({'encounter_id': train['encounter_id'], 'hospital_death': oof_preds})
oof.head()

,encounter_id,hospital_death
0,66154,0.086371
1,114252,0.552521
2,119783,0.000991
3,79267,0.040329
4,92056,0.008799


In [565]:
# file name
model = 'mean_'
perf  = str(round(roc_auc_score(y, oof.hospital_death), 6))[2:7]
name  = model + str(len(features)) + '_' + perf
name

'mean_6_90852'

In [566]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(preds_test.reshape(-1,1))
preds_test = scaler.transform(preds_test.reshape(-1,1)).reshape(-1)

In [567]:
# export OOF preds
oof.to_csv('../oof_preds/' + str(name) + '.csv', index = False)


# export test preds
sub = pd.DataFrame({'encounter_id': test['encounter_id'], 'hospital_death': preds_test})
sub.to_csv('../submissions/' + str(name) + '.csv', index = False)
sub.head()

,encounter_id,hospital_death
0,2,0.015398
1,5,0.029296
2,7,0.013462
3,8,0.157339
4,10,0.604975
